# 4. Make Detections with Model

In [ ]:
import pickle
import numpy as np
import pandas as pd
import cv2
import os
from ultralytics import YOLO


In [ ]:
#Load trained model
with open('body_language_rf.pkl', 'rb') as f:
    model = pickle.load(f)

Enhance video quality for detection


In [ ]:
def enhance_image(image):
    """Enhances a single image using the provided processing steps."""
    # Resize image to 640x640 pixels
    #image = cv2.resize(image, (640, 640))

    # Apply Gaussian Blur to reduce noise
    blurred_color = cv2.GaussianBlur(image, (5, 5), 0)

    # Apply Unsharp Masking
    gaussian_blurred_color = cv2.GaussianBlur(blurred_color, (9, 9), 10.0)
    unsharp_color = cv2.addWeighted(blurred_color, 1.5, gaussian_blurred_color, -0.5, 0)

    # Apply Laplacian sharpening to the grayscale version, then combine with color
    laplacian_color = cv2.Laplacian(cv2.cvtColor(unsharp_color, cv2.COLOR_BGR2GRAY), cv2.CV_64F)
    laplacian_color = cv2.convertScaleAbs(laplacian_color)
    laplacian_color_bgr = cv2.cvtColor(laplacian_color, cv2.COLOR_GRAY2BGR)

    # Combine the unsharp and Laplacian sharpened images
    final_color_sharpened = cv2.addWeighted(unsharp_color, 0.7, laplacian_color_bgr, 0.3, 0)

    # Apply CLAHE to each channel separately
    lab_image = cv2.cvtColor(final_color_sharpened, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    enhanced_lab_image = cv2.merge((l, a, b))
    enhanced_color_image = cv2.cvtColor(enhanced_lab_image, cv2.COLOR_LAB2BGR)

    return enhanced_color_image


Code for detection of persons using Gait and facial recognition

In [ ]:
cap = cv2.VideoCapture(0)
landmark_visibility_threshold = 0.8
conf_thresh=0.5
conf_thresh2=0.5

# Load the YOLO model
model_1 = YOLO("C:\\Users\\FAVOUR\\Downloads\\best(2).pt")  # Replace with trained yolo model
classes=['Teo','Ama','John','-','-'] #Input classes as in ymal file during training using format

# Frame skipping parameter
frame_skip = 5
frame_count = 0

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames
        frame_count+=1000
        #frame_skip+=4
        if frame_count % frame_skip != 0:
            continue
        # Recolor Feed
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False 
        image = enhance_image(img)   

        # Run inference on the frame
        results_1 = model_1(image)  
        
        # Make Detections
        results = holistic.process(image)        
        id= str(results.pose_landmarks)        
        no_of_people= len(list((results_1[0].boxes.cls)))
        print("the number of people is:", {no_of_people})
                

        if not id == "None":            
            #Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            #Index for right body part
            right_elbow_index = mp_pose.PoseLandmark.RIGHT_ELBOW
            right_shoulder_index = mp_pose.PoseLandmark.RIGHT_SHOULDER
            right_hip_index = mp_pose.PoseLandmark.RIGHT_HIP
            right_wrist_index = mp_pose.PoseLandmark.RIGHT_WRIST
            right_heel_index = mp_pose.PoseLandmark.RIGHT_HEEL
            right_knee_index = mp_pose.PoseLandmark.RIGHT_KNEE
            right_ankle_index = mp_pose.PoseLandmark.RIGHT_ANKLE
            
            #Index for left body part
            left_elbow_index = mp_pose.PoseLandmark.LEFT_ELBOW
            left_shoulder_index = mp_pose.PoseLandmark.LEFT_SHOULDER
            left_hip_index = mp_pose.PoseLandmark.LEFT_HIP
            left_wrist_index = mp_pose.PoseLandmark.LEFT_WRIST
            left_heel_index = mp_pose.PoseLandmark.LEFT_HEEL
            left_knee_index = mp_pose.PoseLandmark.LEFT_KNEE
            left_ankle_index = mp_pose.PoseLandmark.LEFT_ANKLE
            # Get the x and y coordinates of the left body part
            left_elbow_cord= (int(results.pose_landmarks.landmark[left_elbow_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_elbow_index].y * image.shape[0]), 0)
            left_shoulder_cord= (int(results.pose_landmarks.landmark[left_shoulder_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_shoulder_index].y * image.shape[0]),0)
            left_hip_cord= (int(results.pose_landmarks.landmark[left_hip_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_hip_index].y * image.shape[0]),0)
            left_wrist_cord= (int(results.pose_landmarks.landmark[left_wrist_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_wrist_index].y * image.shape[0]),0)
            left_heel_cord= (int(results.pose_landmarks.landmark[left_heel_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_heel_index].y * image.shape[0]),0)
            left_knee_cord= (int(results.pose_landmarks.landmark[left_knee_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_knee_index].y * image.shape[0]),0)
            left_ankle_cord= (int(results.pose_landmarks.landmark[left_ankle_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_ankle_index].y * image.shape[0]),0)
            left_hand_cord= (int(results.pose_landmarks.landmark[15].x * image.shape[1]), int(results.pose_landmarks.landmark[15].y * image.shape[0]),0)
            #Get the x and y coordinates of the right body part
            right_elbow_cord= (int(results.pose_landmarks.landmark[right_elbow_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_elbow_index].y * image.shape[0]), 0)
            right_shoulder_cord= (int(results.pose_landmarks.landmark[right_shoulder_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_shoulder_index].y * image.shape[0]),0)
            right_hip_cord= (int(results.pose_landmarks.landmark[right_hip_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_hip_index].y * image.shape[0]),0)
            right_wrist_cord= (int(results.pose_landmarks.landmark[right_wrist_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_wrist_index].y * image.shape[0]),0)
            right_heel_cord= (int(results.pose_landmarks.landmark[right_heel_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_heel_index].y * image.shape[0]),0)
            right_knee_cord= (int(results.pose_landmarks.landmark[right_knee_index].x * image.shape[1]), int(results.pose_landmarks.landmark[right_knee_index].y * image.shape[0]),0)
            right_ankle_cord= (int(results.pose_landmarks.landmark[right_ankle_index].x * image.shape[1]), int(results.pose_landmarks.landmark[left_ankle_index].y * image.shape[0]),0)
            right_hand_cord= (int(results.pose_landmarks.landmark[16].x * image.shape[1]), int(results.pose_landmarks.landmark[16].y * image.shape[0]),0)
            #Calculate the angles 
            right_arm_angle2 = calculateAngle(right_shoulder_cord, right_elbow_cord, right_hip_cord)
            left_arm_angle2= calculateAngle(left_shoulder_cord, left_elbow_cord, left_hip_cord)
            right_arm_angle1=calculateAngle(right_shoulder_cord, right_elbow_cord, right_wrist_cord)
            left_arm_angle1=calculateAngle(left_shoulder_cord, left_elbow_cord, left_wrist_cord)
            right_leg_angle1 =calculateAngle(right_shoulder_cord, right_hip_cord, right_knee_cord)
            left_leg_angle1 =calculateAngle(left_shoulder_cord, left_hip_cord, left_knee_cord)
            right_leg_angle2 =calculateAngle(right_hip_cord, right_knee_cord, right_ankle_cord)
            left_leg_angle2 =calculateAngle(left_hip_cord, left_knee_cord, left_ankle_cord)

            #distance between the heels
            heel_dist = distance_between_landmarks(right_heel_cord, left_heel_cord)
                                                
            #distance between the wrists
            wrist_dist= distance_between_landmarks(right_wrist_cord, left_wrist_cord)
        
            #distance between the knee
            knee_dist= distance_between_landmarks(right_knee_cord, left_knee_cord)
            #height
            height= distance_between_landmarks(right_elbow_cord,right_heel_cord)
            #reference point
            center =  calculate_square_center(right_elbow_cord,left_elbow_cord,right_hip_cord,left_hip_cord)  
            R_heel_ref= distance_between_landmarks(left_heel_cord, center)  
            L_heel_ref= distance_between_landmarks(right_heel_cord,center)
            R_hand_ref= distance_between_landmarks(right_hand_cord, center)
            L_hand_ref= distance_between_landmarks(left_hand_cord, center)
            
            #get view
            if all(landmark.visibility > landmark_visibility_threshold for landmark in results.pose_landmarks.landmark):
                viewpoint =infer_viewpoint(results.pose_landmarks)

                #Prepare Inference dataset for prediction
                cols=['view','heel_dist','wrist_dist','knee_dist','right_arm_angle1','left_arm_angle1','right_arm_angle2','left_arm_angle2','right_leg_angle1','left_leg_angle1','right_leg_angle2','left_leg_angle2','Height','R_hand_ref','L_hand_ref', 'R_heel_ref','L_heel_ref']                
                l1=[viewpoint,heel_dist,wrist_dist,knee_dist,right_arm_angle1,left_arm_angle1,right_arm_angle2,left_arm_angle2,right_leg_angle1,left_leg_angle1,right_leg_angle2,left_leg_angle2,height,R_hand_ref,L_hand_ref,R_heel_ref,L_heel_ref]            
                row =l1 
                X = pd.DataFrame([row])
                X.columns = cols                
                if X['view'][0] == "Front":
                    X["Front"] = 1
                    X["Side"] = 0
                    first_column1 = X.pop('Front')
                    first_column2 = X.pop('Side')
                    X.insert(1, 'Front', first_column1)
                    X.insert(2, 'Side', first_column2)
                    X=X.drop('view', axis=1)
                    X.columns = X.columns.astype(str)
                elif X['view'][0]=="Side":
                    X["Front"] = 0
                    X["Side"] = 1
                    first_column1 = X.pop('Front')
                    first_column2 = X.pop('Side')
                    X.insert(1, 'Front', first_column1)
                    X.insert(2, 'Side', first_column2)
                    X=X.drop('view', axis=1)                
                else:
                    print("No view in DF")

                #Make prediction
                body_language_class = model.predict(X)[0]                
                body_language_prob = model.predict_proba(X)[0]
                a= round(body_language_prob[np.argmax(body_language_prob)],2)                
                
                # Process detections ONLY if there are any
                if results_1[0].boxes is not None and len(results_1[0].boxes.xyxy) > 0: #Check if detections are empty
                    boxes = results_1[0].boxes.xyxy.int().tolist()  # Bounding boxes in xyxy format
                    confidences = results_1[0].boxes.conf.tolist()
                    class_ids = results_1[0].boxes.cls.int().tolist()                    
                    class_id= list((results_1[0].boxes.cls))                   
                    conf= list((results_1[0].boxes.conf))                    
                    predicted_class=[]
                    if not len(class_id) == 0:
                        for i in class_id:
                            i= int(i)
                            predicted_class.append(classes[i])
                    else:
                        print("no class id")

                    cord_list= list((results_1[0].boxes.xyxyn))
                    result= results_1[0].boxes.xyxyn[0].cpu().numpy()
                    x1, y1, x2, y2 = result[0],result[1],result[2],result[3]
                    

                    for i,j,k in zip(cord_list, range(len(classes)), range(len(conf))):
                        frame_height, frame_width = frame.shape[:2]

                        # Convert normalized coordinates to pixel coordinates
                        x1= i[0]*frame_width
                        y1=i[1]*frame_height
                        x2=i[2]*frame_width
                        y2=i[3]*frame_height                        
                        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])# Ensure coordinates are integers
                        
                        # Ensure coordinates are within image boundaries
                        x1 = max(0, x1)
                        y1 = max(0, y1)
                        x2 = min(frame.shape[1], x2)
                        y2 = min(frame.shape[0], y2)
                        name= predicted_class[j]
                        confidence= float(conf[k])
                        person= body_language_class
                        print(name,person)
                        if confidence >conf_thresh or round(body_language_prob[np.argmax(body_language_prob)],2)> conf_thresh2 :                         
                            
                            if round(body_language_prob[np.argmax(body_language_prob)],2)> conf_thresh2:
                                if not person == name:                               
                                    cv2.rectangle(frame, (left_shoulder[0]-60, left_shoulder[1]-80), (right_heel[0]+60, right_heel[1]), (0, 255, 0), 2) #Draws a normal rectangle
                                    cv2.putText(frame, person.split(' ')[0]
                                                    , (left_shoulder[0]-60,(left_shoulder[1]-80)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                                    cv2.putText(frame, viewpoint.split(' ')[0]
                                                    , (220,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)                                
                                
                                    cv2.putText(frame, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                                                , ((left_shoulder[0]-60)+60,(left_shoulder[1]-80)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                                else: 
                                    text_bg_color = (0, 128, 0)  # Same as the bounding box color
                                    text_color = (255, 255, 255)  # White text color
                                    font = cv2.FONT_HERSHEY_SIMPLEX
                                    font_scale = 0.5
                                    font_thickness = 1
                                    (person_width, person_height), _ = cv2.getTextSize(person.split(' ')[0], font, font_scale, font_thickness)
                                    person_text_position = (x1,y1-5)
                                    viewpoint_text_position = (220, 40)
                                    probability_text_position = (x1+65,y1-5)

                                    # Calculate the size of the text to draw the background rectangle accordingly
                                    (person_width, person_height), _ = cv2.getTextSize(name.split(' ')[0], font, font_scale, font_thickness)
                                    (viewpoint_width, viewpoint_height), _ = cv2.getTextSize(viewpoint.split(' ')[0], font, font_scale, font_thickness)
                                    (probability_width, probability_height), _ = cv2.getTextSize(str(round(confidence, 2)), font, font_scale, font_thickness)

                                    # Draw a filled rectangle behind the text to enhance visibility
                                    cv2.rectangle(frame, (person_text_position[0] - 5, person_text_position[1] - person_height - 5), 
                                                (person_text_position[0] + person_width + 5, person_text_position[1] + 5), text_bg_color, -1)
                                    cv2.rectangle(frame, (viewpoint_text_position[0] - 5, viewpoint_text_position[1] - viewpoint_height - 5), 
                                                (viewpoint_text_position[0] + viewpoint_width + 5, viewpoint_text_position[1] + 5), text_bg_color, -1)
                                    cv2.rectangle(frame, (probability_text_position[0] - 5, probability_text_position[1] - probability_height - 5), 
                                                (probability_text_position[0] + probability_width + 5, probability_text_position[1] + 5), text_bg_color, -1)
                                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)                               
                                    cv2.putText(frame, name.split(' ')[0]
                                                    , (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)                                
                                    cv2.putText(frame, str(round(confidence,2))
                                                , (x1+65,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

                            elif confidence >conf_thresh:
                                text_bg_color = (0, 128, 0)  # Same as the bounding box color
                                text_color = (255, 255, 255)  # White text color
                                font = cv2.FONT_HERSHEY_SIMPLEX
                                font_scale = 0.5
                                font_thickness = 1
                                (person_width, person_height), _ = cv2.getTextSize(person.split(' ')[0], font, font_scale, font_thickness)
                                person_text_position = (x1,y1-5)
                                viewpoint_text_position = (220, 40)
                                probability_text_position = (x1+65,y1-5)

                                # Calculate the size of the text to draw the background rectangle accordingly
                                (person_width, person_height), _ = cv2.getTextSize(name.split(' ')[0], font, font_scale, font_thickness)
                                (viewpoint_width, viewpoint_height), _ = cv2.getTextSize(viewpoint.split(' ')[0], font, font_scale, font_thickness)
                                (probability_width, probability_height), _ = cv2.getTextSize(str(round(confidence, 2)), font, font_scale, font_thickness)

                                # Draw a filled rectangle behind the text to enhance visibility
                                cv2.rectangle(frame, (person_text_position[0] - 5, person_text_position[1] - person_height - 5), 
                                            (person_text_position[0] + person_width + 5, person_text_position[1] + 5), text_bg_color, -1)
                                cv2.rectangle(frame, (viewpoint_text_position[0] - 5, viewpoint_text_position[1] - viewpoint_height - 5), 
                                            (viewpoint_text_position[0] + viewpoint_width + 5, viewpoint_text_position[1] + 5), text_bg_color, -1)
                                cv2.rectangle(frame, (probability_text_position[0] - 5, probability_text_position[1] - probability_height - 5), 
                                            (probability_text_position[0] + probability_width + 5, probability_text_position[1] + 5), text_bg_color, -1)
                                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) #Draws a normal rectangle                               
                                cv2.putText(frame, name.split(' ')[0]
                                                , (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)                                
                                cv2.putText(frame, str(round(confidence,2))
                                            , (x1+65,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                            else: 
                                print("low confidence")
                        else: 
                            print("No confidence")    
            else:
                print("no viewpoint due to insufficienct landmarks")
                # Process detections ONLY if there are any
                if results_1[0].boxes is not None and len(results_1[0].boxes.xyxy) > 0: #Check if detections are empty
                    boxes = results_1[0].boxes.xyxy.int().tolist()  # Bounding boxes in xyxy format
                    confidences = results_1[0].boxes.conf.tolist()
                    class_ids = results_1[0].boxes.cls.int().tolist()

                    # print(results[0].boxes.cls)
                    class_id= list((results_1[0].boxes.cls))
                    conf= list((results_1[0].boxes.conf))
                    predicted_class=[]
                    if not len(class_id) == 0:
                        for i in class_id:
                            i= int(i)
                            predicted_class.append(classes[i])
                    else:
                        print("no class id")
                    cord_list= list((results_1[0].boxes.xyxyn))
                    result= results_1[0].boxes.xyxyn[0].cpu().numpy()
                    x1, y1, x2, y2 = result[0],result[1],result[2],result[3]                 

                    for i,j,k in zip(cord_list, range(len(classes)), range(len(conf))):
                        frame_height, frame_width = frame.shape[:2]

                        # Convert normalized coordinates to pixel coordinates
                        x1= i[0]*frame_width
                        y1=i[1]*frame_height
                        x2=i[2]*frame_width
                        y2=i[3]*frame_height            
                        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])# Ensure coordinates are integers

                        # Ensure coordinates are within image boundaries
                        x1 = max(0, x1)
                        y1 = max(0, y1)
                        x2 = min(frame.shape[1], x2)
                        y2 = min(frame.shape[0], y2)
                        name= predicted_class[j]                                           
                        confidence= float(conf[k])
                        if  confidence > conf_thresh:
                            text_bg_color = (0, 128, 0)  # Same as the bounding box color
                            text_color = (255, 255, 255)  # White text color
                            font = cv2.FONT_HERSHEY_SIMPLEX
                            font_scale = 0.5
                            font_thickness = 1
                            (person_width, person_height), _ = cv2.getTextSize(name.split(' ')[0], font, font_scale, font_thickness)
                            person_text_position = (x1,y1-5)
                            viewpoint_text_position = (220, 40)
                            probability_text_position = (x1+65,y1-5)

                            # Calculate the size of the text to draw the background rectangle accordingly
                            (person_width, person_height), _ = cv2.getTextSize(name.split(' ')[0], font, font_scale, font_thickness)                            
                            (probability_width, probability_height), _ = cv2.getTextSize(str(round(confidence, 2)), font, font_scale, font_thickness)

                            # Draw a filled rectangle behind the text to enhance visibility
                            cv2.rectangle(frame, (person_text_position[0] - 5, person_text_position[1] - person_height - 5), 
                                        (person_text_position[0] + person_width + 5, person_text_position[1] + 5), text_bg_color, -1)
                            cv2.rectangle(frame, (probability_text_position[0] - 5, probability_text_position[1] - probability_height - 5), 
                                        (probability_text_position[0] + probability_width + 5, probability_text_position[1] + 5), text_bg_color, -1)
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) #Draws a normal rectangle                               
                            cv2.putText(frame, name.split(' ')[0]
                                            , (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)                                
                            cv2.putText(frame, str(round(confidence,2))
                                        , (x1+65,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                        else:
                            print("low confidence level_2")
                else:
                    print("no facial detection")        
                    
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

         # Process detections ONLY if there are any
        elif results_1[0].boxes is not None and len(results_1[0].boxes.xyxy) > 0: #Check if detections are empty
            boxes = results_1[0].boxes.xyxy.int().tolist()  # Bounding boxes in xyxy format
            confidences = results_1[0].boxes.conf.tolist()
            class_ids = results_1[0].boxes.cls.int().tolist()            
            class_id= list((results_1[0].boxes.cls))
            conf= list((results_1[0].boxes.conf))
            predicted_class=[]
            if not len(class_id) == 0:
                for i in class_id:
                    i= int(i)
                    predicted_class.append(classes[i])
            else:
                print("no class id")
            cord_list= list((results_1[0].boxes.xyxyn))
            result= results_1[0].boxes.xyxyn[0].cpu().numpy()
            x1, y1, x2, y2 = result[0],result[1],result[2],result[3]          

            for i,j,k in zip(cord_list, range(len(classes)), range(len(conf))):
                frame_height, frame_width = frame.shape[:2]

                # Convert normalized coordinates to pixel coordinates
                x1= i[0]*frame_width
                y1=i[1]*frame_height
                x2=i[2]*frame_width
                y2=i[3]*frame_height                
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])# Ensure coordinates are integers

                # Ensure coordinates are within image boundaries
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(frame.shape[1], x2)
                y2 = min(frame.shape[0], y2)
                name= predicted_class[j]                                           
                confidence= float(conf[k])
                if  confidence > conf_thresh:
                    text_bg_color = (0, 128, 0)  # Same as the bounding box color
                    text_color = (255, 255, 255)  # White text color
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 0.5
                    font_thickness = 1
                    (person_width, person_height), _ = cv2.getTextSize(name.split(' ')[0], font, font_scale, font_thickness)
                    person_text_position = (x1,y1-5)
                    viewpoint_text_position = (220, 40)
                    probability_text_position = (x1+65,y1-5)

                    # Calculate the size of the text to draw the background rectangle accordingly
                    (person_width, person_height), _ = cv2.getTextSize(name.split(' ')[0], font, font_scale, font_thickness)
                    (viewpoint_width, viewpoint_height), _ = cv2.getTextSize(viewpoint.split(' ')[0], font, font_scale, font_thickness)
                    (probability_width, probability_height), _ = cv2.getTextSize(str(round(confidence, 2)), font, font_scale, font_thickness)

                    # Draw a filled rectangle behind the text to enhance visibility
                    cv2.rectangle(frame, (person_text_position[0] - 5, person_text_position[1] - person_height - 5), 
                                (person_text_position[0] + person_width + 5, person_text_position[1] + 5), text_bg_color, -1)
                    cv2.rectangle(frame, (viewpoint_text_position[0] - 5, viewpoint_text_position[1] - viewpoint_height - 5), 
                                (viewpoint_text_position[0] + viewpoint_width + 5, viewpoint_text_position[1] + 5), text_bg_color, -1)
                    cv2.rectangle(frame, (probability_text_position[0] - 5, probability_text_position[1] - probability_height - 5), 
                                (probability_text_position[0] + probability_width + 5, probability_text_position[1] + 5), text_bg_color, -1)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) #Draws a normal rectangle                               
                    cv2.putText(frame, name.split(' ')[0]
                                    , (x1,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)                                
                    cv2.putText(frame, str(round(confidence,2))
                                , (x1+65,y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                else:
                    print("low confidence level_3")
            else:
                print("no facial detection")     
        
        # Display the annotated frame
        cv2.imshow('YOLOv8', frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break             
        else: 
            print("no detections available")

cap.release()
cv2.destroyAllWindows()